In [ ]:
%matplotlib inline

from quick_pp.objects import Project

# Load well from saved file
project_name = "VOLVE"
project_path = rf"data\04_project\{project_name}.qppp"
project = Project().load(project_path)
project.get_well_names()

# Quick PP Interpretation

In [ ]:
from quick_pp.ressum import cutoffs_analysis
from quick_pp.machine_learning.feature_engineering import tight_streak_flagging, coal_flagging

# Plot individual results
final_df = project.get_all_data()
final_df['DTC'] = final_df.DT
final_df['BVW'] = final_df.SWT * final_df.PHIE
final_df['VHC'] = (final_df.PHIT * (1 - final_df.SWT)).clip(0, 1)

# Flag tight streaks and coal
final_df['TIGHT_FLAG'] = tight_streak_flagging(final_df.RHOB)
final_df['COAL_FLAG'] = coal_flagging(final_df.NPHI, final_df.RHOB)

final_df['VSHALE'] = final_df.VCLAY
cutoffs = cutoffs_analysis(final_df, percentile=95)

In [ ]:
import pandas as pd
import os

from quick_pp.plotter.plotter import plotly_log
from quick_pp.ressum import calc_reservoir_summary

args = {
    'ressum_cutoffs': dict(
        VSHALE=.5,
        PHIT=0,
        SWT=1
    )
}
output_folder = r'data\04_project\outputs'
ressum_merged = pd.DataFrame()
final_df['ZONES'] = final_df.ZONES.ffill()
for well, data in final_df.groupby('WELL_NAME'):
    fig = plotly_log(data, well_name=well, depth_uom='m', column_widths=[1, 1, 1, 1, 1, 1, .3, 1, 1])
    # fig.show(config=dict(scrollZoom=True))
    fig.write_html(rf"{output_folder}\{well}_log.html", config=dict(scrollZoom=True))

    ressum_df = calc_reservoir_summary(
        data.DEPTH, data.VCLAY, data.PHIT, data.SWT, data.PERM,
        zones=data['ZONES'], cutoffs=args['ressum_cutoffs']
    )
    ressum_df.insert(0, 'WELL_NAME', well)
    ressum_merged = pd.concat([ressum_merged, ressum_df])
ressum_merged.to_csv(os.path.join(output_folder, f'{project_name}_ressum.csv'), index=False)   

In [ ]:
from quick_pp.qaqc import quick_compare

compare_df, fig = quick_compare(final_df, return_fig=True)